### Exercise 1.29.  

> Simpson's Rule is a more accurate method of numerical integration than the method illustrated above. Using Simpson's Rule, the integral of a function f between a and b is approximated as
> $$ \frac{h}{3}[y_0+4y_1+2y_2+4y_3+2y_4+\cdots+2y_{n-1}+4y_{n-1}+y_n]$$
> where $h = (b - a)/n$, for some even integer $n$, and $y_k = f(a + kh)$. (Increasing n increases the accuracy of the approximation.) Define a procedure that takes as arguments f, a, b, and n and returns the value of the integral, computed using Simpson's Rule. Use your procedure to integrate cube between 0 and 1 (with n = 100 and n = 1000), and compare the results to those of the integral procedure shown above.


In [7]:
(define (cube x) (* x x x))


(define (sum term a next b)
  (if (> a b)
      0
      (+ (term a)
         (sum term (next a) next b))))

(define (integral f a b n)
  (define (h)
    (/ (- b a) n))
  (define (y k)
    (f (+ a (* k (h)))))
  (define (plustwo n) (+ n 2))
  (* (/ (h) 3)
     (+ (y 0)
        (y n)
        (* 2 (sum y 2 plustwo (- n 2)))
        (* 4 (sum y 1 plustwo (- n 1))))))
           
           
(integral cube 0 1 100)


1/4

In [8]:
(integral cube 0 1 1000)

1/4

### Exercise 1.30.  

The sum procedure above generates a linear recursion. The procedure can be rewritten so that the sum is performed iteratively. Show how to do this by filling in the missing expressions in the following definition:

```
(define (sum term a next b)
  (define (iter a result)
    (if <??>
        <??>
        (iter <??> <??>)))
  (iter <??> <??>))
```


In [50]:
(define (sumi term a next b)
  (define (iter a result)
    (if (> a b)
        result
        (iter (next a) (+ result (term a)))))
  (iter a 0))

### Exercise 1.31.   

>The sum procedure is only the simplest of a vast number of similar abstractions that can be captured as higher-order procedures. Write an analogous procedure called product that returns the product of the values of a function at points over a given range. 




In [51]:
(define (prod term a next b)
  (if (> a b)
      1
      (* (term a)
         (prod term (next a) next b))))

> Show how to define factorial in terms of product. 

In [52]:
(define (fact n) 
  (define (ident n) n)
  (define (inc n) (+ n 1))
  (prod ident 1 inc n))


> Also use product to compute approximations to  using the formula
> $$ \frac{\pi}{4}=\frac{2\cdot4\cdot4\cdot6\cdot6\cdot8\cdots}{3\cdot3\cdot5\cdot5\cdot7\cdot7\cdots}$$




In [53]:
(define (pi-prod n)
  (define (odd-pi-term x)
    (/ (+ x 1) (+ x 2)))
  (define (even-pi-term x)
    (/ (+ x 2) (+ x 1)))
  (define (plustwo x) (+ x 2))
  (* (prod odd-pi-term 1 plustwo n)
     (prod even-pi-term 2 plustwo n)
     4))

(+ 0.0 (pi-prod 100))


3.1570301764551676

> If your product procedure generates a recursive process, write one that generates an iterative process. If it generates an iterative process, write one that generates a recursive process.


In [54]:
(define (prodi term a next b)
  (define (iter a result)
    (if (> a b)
        result
        (iter (next a) (* result (term a)))))
  (iter a 1))

In [63]:
(define (pi-prodi n)
  (define (odd-pi-term x)
    (/ (+ x 1) (+ x 2)))
  (define (even-pi-term x)
    (/ (+ x 2) (+ x 1)))
  (define (plustwo x) (+ x 2))
  (* (prodi odd-pi-term 1 plustwo n)
     (prodi even-pi-term 2 plustwo n)
     4))

(+ 0.0 (pi-prodi 1000))
 

3.1431607055322663

### Exercise 1.32.

> a. Show that sum and product (exercise 1.31) are both special cases of a still more general notion called accumulate that combines a collection of terms, using some general accumulation function:
> 
> `(accumulate combiner null-value term a next b)`
> 
> Accumulate takes as arguments the same term and range specifications as sum and product, together with a combiner procedure (of two arguments) that specifies how the current term is to be combined with the accumulation of the preceding terms and a null-value that specifies what base value to use when the terms run out. Write accumulate and show how sum and product can both be defined as simple calls to accumulate.



In [69]:
(define (accumulate combiner null-value term a next b)
    (if (> a b)
      null-value
      (combiner (term a)
         (accumulate combiner null-value term (next a) next b))))

In [72]:
(define (sum-acc term a next b)
  (accumulate + 0 term a next b))

(define (prod-acc term a next b)
  (accumulate * 1 term a next b))

> If your accumulate procedure generates a recursive process, write one that generates an iterative process. If it generates an iterative process, write one that generates a recursive process.

In [75]:
(define (accumulatei combiner null-value term a next b)
  (define (iter a result)
    (if (> a b)
        result
        (iter (next a) (combiner result (term a)))))
  (iter a null-value))

### Exercise 1.33.  

> You can obtain an even more general version of accumulate (exercise 1.32) by introducing the notion of a filter on the terms to be combined. That is, combine only those terms derived from values in the range that satisfy a specified condition. The resulting filtered-accumulate abstraction takes the same arguments as accumulate, together with an additional predicate of one argument that specifies the filter. Write filtered-accumulate as a procedure.


In [79]:
(define (filtered-accumulate combiner null-value term a next b filter)
    (if (> a b)
      null-value
      (combiner (if (filter a) (term a) null-value)
         (filtered-accumulate combiner null-value term (next a) next b filter))))

> Show how to express the following using filtered-accumulate
> the sum of the squares of the prime numbers in the interval a to b (assuming that you have a prime? predicate already written)

In [83]:
(define (square x) (* x x))

(define (smallest-divisor n)
  (find-divisor n 2))
(define (find-divisor n test-divisor)
  (cond ((> (square test-divisor) n) n)
        ((divides? test-divisor n) test-divisor)
        (else (find-divisor n (+ test-divisor 1)))))
(define (divides? a b)
  (= (remainder b a) 0))

(define (prime? n)
  (= n (smallest-divisor n)))

(define (square n)
  (* n n))

(define (sum-square-prime a b)
    (filtered-accumulate + 0 square a inc b prime?))

> the product of all the positive integers less than n that are relatively prime to n (i.e., all positive integers i < n such that GCD(i,n) = 1).

In [90]:
(define (gcd a b)
  (if (= b 0)
      a
      (gcd b (remainder a b))))

(define (relprime a b)
  (= 1 (gcd a b)))

(define (prod-pos-rel-p n)
  (define (filt a) (relprime n a))
  (filtered-accumulate * 1 ident 1 inc n filt))

### Exercise 1.34.  

> Suppose we define the procedure

> ```scheme 
(define (f g)
  (g 2))
```
> Then we have
> ```scheme
(f square)
4
(f (lambda (z) (* z (+ z 1))))
6
```

> What happens if we (perversely) ask the interpreter to evaluate the combination (f f)? Explain.


The evaluation ends up looking something like:

```scheme
(f f)
(f 2)
(2 2) ; error
```


### Exercise 1.35.  

> Show that the golden ratio $\phi$ is a fixed point of the transformation $x \rightarrow 1 + 1/x$ , and use this fact to compute $\phi$ by means of the fixed-point procedure.

$$1 + \frac{1}{\frac{1+\sqrt 5}{2}} = 1 + \frac{2}{1+\sqrt 5}=\frac{1+\sqrt 5 + 2}{1 + \sqrt 5} = \frac{3+\sqrt 5}{1+\sqrt 5}=\frac{(3+\sqrt 5)(1-\sqrt 5)}{-4}=\frac{3-2\sqrt 5 - 5}{-4}=\frac{-2-2\sqrt 5}{-4}=\frac{1+\sqrt 5}{2}$$

In [17]:
(define tolerance 0.00001)
(define (fixed-point f first-guess)
  (define (close-enough? v1 v2)
    (< (abs (- v1 v2)) tolerance))
  (define (t guess)
    (let ((next (f guess)))
      (if (close-enough? guess next)
          next
          (t next))))
  (t first-guess))

In [19]:
(fixed-point (lambda (x) (+ 1.0 (/ 1.0 x))) 1 )

1.6180327868852458

### Exercise 1.36.  

> Modify fixed-point so that it prints the sequence of approximations it generates, using the newline and display primitives shown in exercise 1.22. 




In [33]:
(define (fixed-point-prt f first-guess)
  (define (prt-guess n)
    (newline)
    (display n))
  (define (close-enough? v1 v2)
    (< (abs (- v1 v2)) tolerance))
  (define (t guess)
    (prt-guess guess)
    (let ((next (f guess)))
      (if (close-enough? guess next)
          next
          (t next))))
  (t first-guess))


> Then find a solution to $x^x = 1000$ by finding a fixed point of $x \rightarrow \log(1000)/\log(x)$. (Use Scheme's primitive log procedure, which computes natural logarithms.) Compare the number of steps this takes with and without average damping. (Note that you cannot start fixed-point with a guess of 1, as this would cause division by log(1) = 0.)



In [32]:
(import-from "math" "log")

(fixed-point-prt (lambda (x) (/ (log 1000) (log x))) 1.5)


1.5
17.036620761802716
2.436284152826871
7.7573914048784065
3.3718636013068974
5.683217478018266
3.97564638093712
5.004940305230897
4.2893976408423535
4.743860707684508
4.437003894526853
4.6361416205906485
4.503444951269147
4.590350549476868
4.532777517802648
4.570631779772813
4.545618222336422
4.562092653795064
4.551218723744055
4.558385805707352
4.553657479516671
4.55677495241968
4.554718702465183
4.556074615314888
4.555180352768613
4.555770074687025
4.555381152108018
4.555637634081652
4.555468486740348
4.555580035270157
4.555506470667713
4.555554984963888
4.5555229906097905
4.555544090254035
4.555530175417048

4.555539351985717

In [35]:
(define (average x y) (+ (/ x 2) (/ y 2)))
(fixed-point-prt (lambda (x) (average x (/ (log 1000) (log x)))) 1.5)


1.5
9.268310380901358
6.185343522487719
4.988133688461795
4.643254620420954
4.571101497091747
4.5582061760763715
4.555990975858476
4.555613236666653
4.555548906156018
4.555537952796512

4.555536087870658

### Exercise 1.37. 

> An infinite continued fraction is an expression of the form $$f=\frac{N_1}{D_1+\frac{N_2}{D_2+\frac{N_3}{D_3+\cdots}}}$$
> As an example, one can show that the infinite continued fraction expansion with the $N_i$ and the $D_i$ all equal to 1 produces $1/\phi$, where  is the golden ratio (described in section 1.2.2). One way to approximate an infinite continued fraction is to truncate the expansion after a given number of terms. Such a truncation -- a so-called k-term finite continued fraction -- has the form $$\frac{N_1}{D_1+\frac{N_2}{\ddots+\frac{N_K}{D_K}}}$$
> Suppose that n and d are procedures of one argument (the term index i) that return the $N_i$ and $D_i$ of the terms of the continued fraction. Define a procedure cont-frac such that evaluating `(cont-frac n d k)` computes the value of the k-term finite continued fraction. 





In [45]:
(define (cont-frac n d k)
  (define (rec i)
    (/ (n i)
       (+ (d i) 
          (if (= i k)
              0
              (rec (+ i 1))))))
  (rec 1))

> Check your procedure by approximating $1/\phi$ using
> 
```scheme
(cont-frac (lambda (i) 1.0)
           (lambda (i) 1.0)
           k)
```
> for successive values of k. How large must you make k in order to get an approximation that is accurate to 4 decimal places?

In [44]:
(cont-frac (lambda (i) 1.0)
           (lambda (i) 1.0)
           5)

0.625

In [46]:
(cont-frac (lambda (i) 1.0)
           (lambda (i) 1.0)
           10)

0.6179775280898876

In [43]:
(cont-frac (lambda (i) 1.0)
           (lambda (i) 1.0)
           11)

0.6180555555555556

> If your cont-frac procedure generates a recursive process, write one that generates an iterative process. If it generates an iterative process, write one that generates a recursive process.

In [49]:
(define (cont-frac-iter n d k)
  (define (iter result i)
    (if (< i 1)
        result
        (iter (/ (n i) (+ (d i) result)) (- i 1))))
        
  (iter 0 k))

In [51]:
(cont-frac-iter (lambda (i) 1.0)
                (lambda (i) 1.0)
                11)

0.6180555555555556

### Exercise 1.38.  

> In 1737, the Swiss mathematician Leonhard Euler published a memoir De Fractionibus Continuis, which included a continued fraction expansion for $e - 2$, where $e$ is the base of the natural logarithms. In this fraction, the $N_i$ are all 1, and the $D_i$ are successively 1, 2, 1, 1, 4, 1, 1, 6, 1, 1, 8, .... Write a program that uses your cont-frac procedure from exercise 1.37 to approximate e, based on Euler's expansion.


In [63]:
(define (e-est) 
  (+ 2 (cont-frac-iter (lambda (i) 1)
                       (lambda (d) (if (= 2 (% d 3))
                                       (* 2 (/ (+ d 1) 3))
                                       1.0))
                       15)))

(e-est)
                                       

2.718281828470584

### Exercise 1.39.  

> A continued fraction representation of the tangent function was published in 1770 by the German mathematician J.H. Lambert: $$ \tan x = \frac{x}{1 - \frac{x^2}{3-\frac{x^2}{5-\ddots}}}$$
> where x is in radians. Define a procedure `(tan-cf x k)` that computes an approximation to the tangent function based on Lambert's formula. K specifies the number of terms to compute, as in exercise 1.37.




In [71]:
(define (tan-cf x k)
  (cont-frac-iter (lambda (i) (if (= i 1.0)
                                  x
                                  (- (* x x))))
                  (lambda (i) (- (* 2 i) 1.0))
                  k))

### Exercise 1.40.  

> Define a procedure cubic that can be used together with the newtons-method procedure in expressions of the form
> ```(newtons-method (cubic a b c) 1)```
> to approximate zeros of the cubic $x^3 + ax^2 + bx + c$.



In [80]:
(define (deriv g)
  (lambda (x)
    (/ (- (g (+ x dx)) (g x))
       dx)))

(define dx 0.00001)

(define (newton-transform g)
  (lambda (x)
    (- x (/ (g x) ((deriv g) x)))))

(define (newtons-method g guess)
  (fixed-point (newton-transform g) guess))

(define (cubic a b c)
  (lambda (x)
      (+ (* x x x)
         (* a x x)
         (* b x)
         c)))


### Exercise 1.41.  

> Define a procedure double that takes a procedure of one argument as argument and returns a procedure that applies the original procedure twice. For example, if inc is a procedure that adds 1 to its argument, then `(double inc)` should be a procedure that adds 2. 

In [82]:
(define (double x)
  (lambda (k) (x (x k))))

(define (inc x) (+ x 1))

((double inc) 2)

4

> What value is returned by `(((double (double double)) inc) 5)`

In [83]:
(((double (double double)) inc) 5)

21

### Exercise 1.42. 

> Let f and g be two one-argument functions. The composition f after g is defined to be the function $x\rightarrow f(g(x))$. Define a procedure compose that implements composition. For example, if inc is a procedure that adds 1 to its argument, 
> ```scheme
((compose square inc) 6)
49
```


In [89]:
(define (square x) (* x x))
(define (compose f g) (lambda (x) (f (g x))))

((compose square inc) 6)

49

### Exercise 1.43. 

> If $f$ is a numerical function and $n$ is a positive integer, then we can form the $n$th repeated application of $f$, which is defined to be the function whose value at $x$ is $f(f(\ldots(f(x))\ldots))$. For example, if $f$ is the function $x\rightarrow x + 1$, then the $n$th repeated application of $f$ is the function $x\rightarrow x + n$. If $f$ is the operation of squaring a number, then the $n$th repeated application of $f$ is the function that raises its argument to the $2^n$th power. Write a procedure that takes as inputs a procedure that computes $f$ and a positive integer $n$ and returns the procedure that computes the $n$th repeated application of $f$. Your procedure should be able to be used as follows:
```scheme
((repeated square 2) 5)
625
```


In [92]:
(define (repeated f n)
  (if (= n 1)
      f
      (compose (repeated f (- n 1)) f)))

### Exercise 1.44.  

> The idea of smoothing a function is an important concept in signal processing. If $f$ is a function and $dx$ is some small number, then the smoothed version of $f$ is the function whose value at a point $x$ is the average of $f(x - dx)$, $f(x)$, and $f(x + dx)$. Write a procedure `smooth` that takes as input a procedure that computes $f$ and returns a procedure that computes the smoothed $f$. 

In [103]:
(define (smooth f) 
  (lambda (x)
    (+ (/ (f (- x dx)) 3)
       (/ (f x) 3)
       (/ (f (+ x dx)) 3))))

>It is sometimes valuable to repeatedly smooth a function (that is, smooth the smoothed function, and so on) to obtained the n-fold smoothed function.  Show how to generate the n-fold smoothed function of any given function using smooth and repeated from exercise 1.43.



In [107]:
(define (n-fold-smooth f n)
  ((repeated smooth n) f))

### Exercise 1.45.  

> We saw in section 1.3.3 that attempting to compute square roots by naively finding a fixed point of $y \rightarrow x/y$ does not converge, and that this can be fixed by average damping. The same method works for finding cube roots as fixed points of the average-damped $y\rightarrow x/y^2$. Unfortunately, the process does not work for fourth roots -- a single average damp is not enough to make a fixed-point search for $y\rightarrow  x/y^3$ converge. On the other hand, if we average damp twice (i.e., use the average damp of the average damp of $y\rightarrow x/y^3$) the fixed-point search does converge. 

> Do some experiments to determine how many average damps are required to compute $n$th roots as a fixed-point search based upon repeated average damping of $y\rightarrow x/y^{n-1}$. 

>Use this to implement a simple procedure for computing nth roots using fixed-point, average-damp, and the repeated procedure of exercise 1.43. Assume that any arithmetic operations you need are available as primitives.



In [119]:
(define (average-damp f)
  (lambda (x) (average x (f x))))

(define (fixed-point-of-transform g transform guess)
  (fixed-point (transform g) guess))

(define (cbrt x)
  (fixed-point-of-transform (lambda (y) (/ x (square y)))
                            average-damp
                            1.0))

(cbrt 1000) ; works


10.000002544054729

In [137]:
(define (fourth-root x)
  (fixed-point-of-transform (lambda (y) (/ x (pow y 3)))
                            (repeated average-damp 2)
                            1.0))

(fourth-root 1000) ; works

5.623413251903491

In [138]:
(define (fifth-root x)
  (fixed-point-of-transform (lambda (y) (/ x (pow y 4)))
                            (repeated average-damp 2)
                            1.0))

(fifth-root 100000) ; works

9.99999869212542

In [128]:
(define (sixth-root x)
  (fixed-point-of-transform (lambda (y) (/ x (* y y y y y)))
                            (repeated average-damp 2)
                            1.0))

(sixth-root 10000) ; works

4.641591147676662

In [139]:
(define (seventh-root x)
  (fixed-point-of-transform (lambda (y) (/ x (pow y 6)))
                            (repeated average-damp 2)
                            1.0))

(seventh-root 2) ; works

1.1040857488809646

In [140]:
(define (eighth-root x)
  (fixed-point-of-transform (lambda (y) (/ x (pow y 7)))
                            (repeated average-damp 3)
                            1.0))

(eighth-root 2000000) ; works

6.13237563517304

In [141]:
(define (fifteenth-root x)
  (fixed-point-of-transform (lambda (y) (/ x (pow y 14)))
                            (repeated average-damp 3)
                            1.0))

(fifteenth-root 2000000) ; works

2.630688324520184

In [143]:
(define (sixteenth-root x)
  (fixed-point-of-transform (lambda (y) (/ x (pow y 15)))
                            (repeated average-damp 4)
                            1.0))

(sixteenth-root 2000000) ; works

2.4763633891604453

It appears the floor of $log_2 (n)$ is the necessary number of dampings.

In [161]:
(import-from "math" "floor")

(define (floorlog n) (floor (/ (log n) (log 2))))

(define (nth-root x n)
  (fixed-point-of-transform (lambda (y) (/ x (pow y (- n 1))))
                            (repeated average-damp (floorlog n))
                            1.0))

### Exercise 1.46.  

> Several of the numerical methods described in this chapter are instances of an extremely general computational strategy known as iterative improvement. Iterative improvement says that, to compute something, we start with an initial guess for the answer, test if the guess is good enough, and otherwise improve the guess and continue the process using the improved guess as the new guess. Write a procedure `iterative-improve` that takes two procedures as arguments: a method for telling whether a guess is good enough and a method for improving a guess. `Iterative-improve` should return as its value a procedure that takes a guess as argument and keeps improving the guess until it is good enough. 





In [180]:
(define (iterative-improve good-enough? improve)
  (define (t guess)
    (if (good-enough? guess)
        guess
        (t (improve guess))))
  (lambda (x) (t x)))

> Rewrite the sqrt procedure of section 1.1.7 and the fixed-point procedure of section 1.3.3 in terms of iterative-improve.

In [191]:
(define (sqrt-ii x) 
  ((iterative-improve (lambda (g) (< (abs (- (square g) x)) 0.001))
                     (lambda (g) (average g (/ x g))))
   1.0))

In [192]:
(sqrt-ii 100)

10.000000000139897

In [194]:
(define (fixed-point-ii f first-guess)
    ((iterative-improve (lambda (g) (< (abs (- (f g) g)) tolerance)) f) first-guess))

In [200]:
(fixed-point-ii (lambda (x) (+ 1.0 (/ 1.0 x))) 1 )

1.6180371352785146